In [12]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import re

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arceb\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arceb\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
! pip install nltk

You should consider upgrading via the 'C:\Users\arceb\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
! pip install gensim

You should consider upgrading via the 'C:\Users\arceb\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
encoding_type = 'ISO-8859-1'
df = pd.read_csv(r'C:\Users\arceb\PycharmProjects\Projet_DSSS_2023/fullcorpus20210725_DATE.csv', encoding=encoding_type)
df

,Unnamed: 0,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term
0,1,"Le Figaro, no. 23520",L'agenda cachÃ© de Moscou en Italie,2020-03-30,"Lasserre, Isabelle",LES SYMBOLES comptent parfois plus que les fai...,"Sur les camions militaires qui, en colonnes, ...",Deux semaines pour se ressaisir et s'unir ou p...,Figaro,migrant_migratoire
1,2,Le Monde,RÃ©cit Les trois semaines qui ont chamboulÃ©...,2020-03-27,"Ãric Albert, CÃ©cile Boutelet, JÃ©rÃ´me Gauth...","Londres, Berlin, Rome et Bruxelles corresponda...","- Jeudi 26 mars, les Vingt-Sept devaient se r...","pandÃ©mie touche tout le monde, indiffÃ©remmen...",Monde,migrant_migratoire
2,3,Le Figaro (site web),Ãlargissement de lâUnion europÃ©enne Ã lâ...,2020-03-26,"Tandonnet, Maxime",Fin observateur de la vie politique franÃ§aise...,"contributeur rÃ©gulier du FigaroVox, Maxime Ta...",es yeux et de se prÃ©parer au changement dâu...,Figaro,migrant_migratoire
3,4,Le Monde (site web),CoronavirusÂ : les trois semaines qui ont cham...,2020-03-26,"Eric Albert, JÃ©rÃ´me Gautheret, CÃ©cile Boute...","Jeudi 26Â mars, les Vingt-Sept doivent se rÃ©u...",n visioconfÃ©rence pour la troisiÃ¨me fois en ...,"e la tenue de ses comptes publics, nâest pas...",Monde,migrant_migratoire
4,5,Le Monde (site web),Marc LazarÂ : Â«Â Plus lâItalie fait nation ...,2020-03-25,Marc Lazar,Tribune. LâItalie est le pays qui enregistre...,date du 25Â mars le plus grand nombre de mort...,t de la School of Government de la Luiss (Libr...,Monde,migrant_migratoire
...,...,...,...,...,...,...,...,...,...,...
8021,8035,LibÃ©ration,TUNISIE Ã©vacuation des rÃ©fugiÃ©s,2011-03-04,NaN,Le monde s'organise face Ã la crisedesrÃ©fugi...,"i secoue la Tunisie, que 95 000 personnes ont ...","hÃ©licoptÃ¨res franÃ§ais Mistral, ont Ã©gale...",LibÃ©ration,asylum_refugee
8022,8036,Le Figaro.fr,Libye/UE: 30M d'euros pour les rÃ©fugiÃ©s,2011-03-03,AFP,La Commissaire europÃ©enne Kristalina Georgiev...,"hargÃ©e de la coopÃ©ration internationale, de ...",e Ã Tunis avec le ministre des affaire Ã©tran...,Figaro,asylum_refugee
8023,8037,"La Croix, no. 38872",Courrier,2011-01-18,NaN,rÃ©fugiÃ©s du tchadMerci pour votre article su...,"rÃ©fugiÃ©s dans l'est du Tchad, paru le 27 dÃ...","chaque gÃ©nÃ©ration, on se demande qui, des Fr...",Croix,asylum_refugee
8024,8038,"LibÃ©ration, no. 9175",Reportage L'Ã©trange aller-retour des Karens...,2010-11-11,Arnaud DUBUS,"Ballots sur la tÃªte, les bras chargÃ©s de sac...","de nouilles ou le sac au dos, quelque 15 000 B...",t des Ã©lections du week-end : la junte avait ...,LibÃ©ration,asylum_refugee


In [7]:
df.replace({'Ã©': 'é', 'Ã¨': 'è', 'Ãª': 'ê', 'Ã´': 'ô','Ã': 'à' }, regex=True, inplace=True)
df

,Unnamed: 0,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term
0,1,"Le Figaro, no. 23520",L'agenda caché de Moscou en Italie,2020-03-30,"Lasserre, Isabelle",LES SYMBOLES comptent parfois plus que les fai...,"Sur les camions militaires qui, en colonnes, ...",Deux semaines pour se ressaisir et s'unir ou p...,Figaro,migrant_migratoire
1,2,Le Monde,Récit Les trois semaines qui ont chamboulé l...,2020-03-27,"àric Albert, Cécile Boutelet, Jérôme Gauthere...","Londres, Berlin, Rome et Bruxelles corresponda...","- Jeudi 26 mars, les Vingt-Sept devaient se r...","pandémie touche tout le monde, indifféremment ...",Monde,migrant_migratoire
2,3,Le Figaro (site web),àlargissement de lâUnion européenne à lâ...,2020-03-26,"Tandonnet, Maxime",Fin observateur de la vie politique franà§aise...,"contributeur régulier du FigaroVox, Maxime Tan...",es yeux et de se préparer au changement dâun...,Figaro,migrant_migratoire
3,4,Le Monde (site web),CoronavirusÂ : les trois semaines qui ont cham...,2020-03-26,"Eric Albert, Jérôme Gautheret, Cécile Boutelet...","Jeudi 26Â mars, les Vingt-Sept doivent se réun...",n visioconférence pour la troisième fois en mo...,"e la tenue de ses comptes publics, nâest pas...",Monde,migrant_migratoire
4,5,Le Monde (site web),Marc LazarÂ : Â«Â Plus lâItalie fait nation ...,2020-03-25,Marc Lazar,Tribune. LâItalie est le pays qui enregistre...,date du 25Â mars le plus grand nombre de mort...,t de la School of Government de la Luiss (Libr...,Monde,migrant_migratoire
...,...,...,...,...,...,...,...,...,...,...
8021,8035,Libération,TUNISIE évacuation des réfugiés,2011-03-04,NaN,Le monde s'organise face à la crisedesréfugié...,"i secoue la Tunisie, que 95 000 personnes ont ...","hélicoptères franà§ais Mistral, ont égalemen...",Libération,asylum_refugee
8022,8036,Le Figaro.fr,Libye/UE: 30M d'euros pour les réfugiés,2011-03-03,AFP,La Commissaire européenne Kristalina Georgieva...,"hargée de la coopération internationale, de l'...",e à Tunis avec le ministre des affaire étrang...,Figaro,asylum_refugee
8023,8037,"La Croix, no. 38872",Courrier,2011-01-18,NaN,réfugiés du tchadMerci pour votre article sur ...,"réfugiés dans l'est du Tchad, paru le 27 déce...","chaque génération, on se demande qui, des Fran...",Croix,asylum_refugee
8024,8038,"Libération, no. 9175",Reportage L'étrange aller-retour des Karens ...,2010-11-11,Arnaud DUBUS,"Ballots sur la tête, les bras chargés de sache...","de nouilles ou le sac au dos, quelque 15 000 B...",t des élections du week-end : la junte avait d...,Libération,asylum_refugee


In [8]:
# Assuming 'df' is your DataFrame and the sentence is in a column named 'text'
df.replace({'Â': '', 'â': "'", 'à ': 'à'}, regex=True, inplace=True)
df

,Unnamed: 0,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term
0,1,"Le Figaro, no. 23520",L'agenda caché de Moscou en Italie,2020-03-30,"Lasserre, Isabelle",LES SYMBOLES comptent parfois plus que les fai...,"Sur les camions militaires qui, en colonnes, ...",Deux semaines pour se ressaisir et s'unir ou p...,Figaro,migrant_migratoire
1,2,Le Monde,Récit Les trois semaines qui ont chamboulé l...,2020-03-27,"àric Albert, Cécile Boutelet, Jérôme Gauthere...","Londres, Berlin, Rome et Bruxelles corresponda...","- Jeudi 26 mars, les Vingt-Sept devaient se r...","pandémie touche tout le monde, indifféremment ...",Monde,migrant_migratoire
2,3,Le Figaro (site web),àlargissement de l'Union européenne à l'heur...,2020-03-26,"Tandonnet, Maxime",Fin observateur de la vie politique franà§aise...,"contributeur régulier du FigaroVox, Maxime Tan...",es yeux et de se préparer au changement d'un m...,Figaro,migrant_migratoire
3,4,Le Monde (site web),Coronavirus : les trois semaines qui ont chamb...,2020-03-26,"Eric Albert, Jérôme Gautheret, Cécile Boutelet...","Jeudi 26 mars, les Vingt-Sept doivent se réuni...",n visioconférence pour la troisième fois en mo...,"e la tenue de ses comptes publics, n'est pas s...",Monde,migrant_migratoire
4,5,Le Monde (site web),Marc Lazar : « Plus l'Italie fait nation face ...,2020-03-25,Marc Lazar,Tribune. L'Italie est le pays qui enregistre à...,date du 25 mars le plus grand nombre de morts...,t de la School of Government de la Luiss (Libr...,Monde,migrant_migratoire
...,...,...,...,...,...,...,...,...,...,...
8021,8035,Libération,TUNISIE évacuation des réfugiés,2011-03-04,NaN,Le monde s'organise face à la crisedesréfugié...,"i secoue la Tunisie, que 95 000 personnes ont ...","hélicoptères franà§ais Mistral, ont égalemen...",Libération,asylum_refugee
8022,8036,Le Figaro.fr,Libye/UE: 30M d'euros pour les réfugiés,2011-03-03,AFP,La Commissaire européenne Kristalina Georgieva...,"hargée de la coopération internationale, de l'...",e à Tunis avec le ministre des affaire étrang...,Figaro,asylum_refugee
8023,8037,"La Croix, no. 38872",Courrier,2011-01-18,NaN,réfugiés du tchadMerci pour votre article sur ...,"réfugiés dans l'est du Tchad, paru le 27 déce...","chaque génération, on se demande qui, des Fran...",Croix,asylum_refugee
8024,8038,"Libération, no. 9175",Reportage L'étrange aller-retour des Karens ...,2010-11-11,Arnaud DUBUS,"Ballots sur la tête, les bras chargés de sache...","de nouilles ou le sac au dos, quelque 15 000 B...",t des élections du week-end : la junte avait d...,Libération,asylum_refugee


In [19]:
# Preprocess the text data
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    stop_words = set(stopwords.words('french'))
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords
    return tokens

# Apply preprocessing to the 'Text' column
df['Tokens'] = df['Texte'].apply(preprocess_text)

# Create a dictionary and a corpus
dictionary = corpora.Dictionary(df['Tokens'])
corpus = [dictionary.doc2bow(tokens) for tokens in df['Tokens']]

# Train an LDA model
num_topics = 5  # You can adjust this based on your dataset
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Print topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

# Calculate coherence score for model evaluation
coherence_model = CoherenceModel(model=lda_model, texts=df['Tokens'], dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f'Coherence Score: {coherence_score}')

KeyboardInterrupt: 

In [20]:
# Filter articles containing the term "Africa"
search_term = "Africa"
filtered_articles_africa = df[df['Texte'].str.contains(search_term, case=False)]

# Display the filtered articles
filtered_articles_africa

,Unnamed: 0,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term,Tokens
20,21,Le Figaro (site web),«Vague migratoire: le seul moyen de faire recu...,2020-03-06,"Bonnamy, Jean-Loup",Renaud Girard est correspondant de guerre et c...,niqueur international du Figaro. FIGAROVOX.- P...,"» . Si nous l'avions fait, nous pourrions joue...",Figaro,migrant_migratoire,"[renaud, girard, correspondant, guerre, chroni..."
54,55,Le Monde (site web),Une photo de prétendus « réfugiés européens » ...,2019-10-18,Assma Maad,Des images relayées depuis la crisedesmigrants...,2015 circulent régulièrement depuis sur les r...,"érieur du stade de football local, ont été des...",Monde,migrant_migratoire,"[images, relayes, depuis, crisedesmigrants, ci..."
57,58,Le Figaro (site web),Stephen Smith: «L'Afrique est une salle d'atte...,2019-10-10,"Devecchio, Alexandre",LE FIGARO - «La France ne peut pas accueillir ...,"t le monde», affirme Macron, qui a lancé un dé...",ys doivent avoir le dernier mot chez eux. «Qu...,Figaro,migrant_migratoire,"[figaro, france, peut, accueillir, tout, monde..."
97,98,Le Monde.fr,Bac 2019 histoire-géo : le corrigé de la série S,2019-06-18,NaN,"Après la philosophie, lundi 17 juin, les candi...",s au bac 2019 ont planché sur les épreuves d'h...,aciques et ponctuels; dans la troisième des fi...,Monde,migrant_migratoire,"[aprs, philosophie, lundi, juin, candidats, ba..."
102,103,"Les Echos, no. 22953","L'immigration, un thème de campagne prisé dans...",2019-05-22,CATHERINE CHATIGNOUX,La crisedesmigrants a pourri les quatre derniè...,années de l'Union européenne comme le risque ...,anti-migrants est devenu la principale arme po...,Autre,migrant_migratoire,"[crisedesmigrants, a, pourri, quatre, dernires..."
...,...,...,...,...,...,...,...,...,...,...,...
7929,7943,Le Monde,Reportage Et l'Allemagne ouvrit ses portes ...,2015-09-08,Marion Van renterghem,"Vienne, munich - envoyée spéciale - Alemania! ...","Alemania! » Il est minuit, dimanche 6 septem...",bourg! la Suède! la Norvège! Chacun a ses voeu...,Monde,asylum_refugee,"[vienne, munich, envoye, spciale, alemania, al..."
7969,7983,Les Echos (site web),"Franà§ois Héran : « Sur les réfugiés, l'Europe...",2015-09-03,VIRGINIE ROBERT; ELSA FREYSSENET,"Pensez-vous, comme Angela Merkel, que les bloc...",s de certains pays européens sur les réfugiés ...,t évoque celui des régimes soviétiques : un mo...,Echos,migrant_migratoire,"[pensezvous, comme, angela, merkel, blocages, ..."
7985,7999,"Les Echos, no. 22012",Quand l'Afrique disserte des crises mondiales,2015-08-31,YVES BOURDILLON,"Réchauffement climatique, crisedesréfugiés, gu...",e contre les djihadistes... ce sont des thèmes...,ays le plus riche d'Afrique par le PIB par têt...,Echos,migrant_migratoire,"[rchauffement, climatique, crisedesrfugis, gue..."
8005,8019,"La Croix, no. 40216","CETTE SEMAINE Le 20 juin, l'ONU célèbre comme ...",2015-06-20,JOVANOVIC Pierre,L'àVàNEMENTUne date symbolique.La journée m...,le des réfugiés a été instituée par l'ONU pou...,"giés ayant fui à Malte, en Grèce et en Indoné...",Croix,asylum_refugee,"[lvnementune, date, symboliquela, journe, mond..."


In [22]:
# Filter articles containing the term "Africa"
search_term = "Moyen-Orient"
filtered_articles_mo = df[df['Texte'].str.contains(search_term, case=False)]

# Display the filtered articles
filtered_articles_mo

,Unnamed: 0,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term,Tokens
20,21,Le Figaro (site web),«Vague migratoire: le seul moyen de faire recu...,2020-03-06,"Bonnamy, Jean-Loup",Renaud Girard est correspondant de guerre et c...,niqueur international du Figaro. FIGAROVOX.- P...,"» . Si nous l'avions fait, nous pourrions joue...",Figaro,migrant_migratoire,"[renaud, girard, correspondant, guerre, chroni..."
25,26,"Le Figaro, no. 23497",Il faut défendre les frontières de l'Europe !,2020-03-03,"Girard, Renaud",Les frontières de l'Europe sont attaquées. Dim...,"he 1er mars 2020, de solides jeunes barbus, cr...",ter démocratiquement sur ce point les sociétés...,Figaro,migrant_migratoire,"[frontires, leurope, attaques, dimanche, er, m..."
116,117,Le Monde,Ingérences russes 1 | 6 - Poutine parrain de l...,2019-04-04,Isabelle Mandraud,Moscou correspondante - Le buste penché en ava...,"sur la ­table, Vladimir Poutine darde un regar...","cept de « démocratie souveraine », qui a large...",Monde,migrant_migratoire,"[moscou, correspondante, buste, pench, avant, ..."
167,168,"Le Figaro, no. 23094",Les trouble-fête conviés à la table d'Emmanue...,2018-11-12,"Jaulmes, Adrien",à CHAQUE réunion de famille ses trouble-fête....,mi les quelque soixante-dix chefs d'àtat et d...,onies qui commémoraient la fin de l'un des plu...,Figaro,migrant_migratoire,"[chaque, runion, famille, troublefte, parmi, q..."
188,189,"Le Figaro, no. 23058",Europe : l'élection de tous les dangers,2018-10-01,"Baverez, Nicolas","Cent ans après la fin de la Grande Guerre, l'U...",n européenne affronte une crise existentielle....,n'est pas Emmanuel Macron qui refermera la pa...,Figaro,migrant_migratoire,"[cent, ans, aprs, fin, grande, guerre, lunion,..."
...,...,...,...,...,...,...,...,...,...,...,...
7958,7972,Les Echos (site web),Réfugiés : Poutine favorable à une coalition ...,2015-09-04,CLAUDE FOUQUET,"Sur l'air du « je vous l'avais bien dit » , le...",ésident russe Vladimir Poutine a estimé ce ven...,aussi inutile de parler de rejoindre la coalit...,Echos,migrant_migratoire,"[lair, lavais, bien, dit, prsident, russe, vla..."
7963,7977,Le Monde.fr,Hongrie : des centaines de réfugiés marchent s...,2015-09-04,NaN,Le chaos règne toujours en Hongrie dans la cri...,"esréfugiés. Des centaines d'entre eux, abandon...","t, le cardinal Péter Erdà¶, est sorti de son l...",Monde,asylum_refugee,"[chaos, rgne, toujours, hongrie, crisedesrfugi..."
7964,7978,Le Parisien.fr,EN DIRECT. La Hongrie réquisitionne des cars p...,2015-09-04,V.G. et M-L.W. (avec agences),Deux jours après l'électrochoc provoqué par la...,"blication de la photo du petit Aylan, mort sur...",réfugiés VIDEO. Erdogan : « L'Europe a transf...,Parisien,asylum_refugee,"[deux, jours, aprs, llectrochoc, provoqu, publ..."
8008,8022,Libération,éditorial Tragédie,2015-04-07,Alexandra Schwartzbrod,La mondialisation de l'indifférence. L'express...,avait été utilisée une première fois par le p...,r le ressentiment et la haine. Notamment la Fr...,Libération,asylum_refugee,"[mondialisation, lindiffrence, lexpression, ut..."


In [24]:
# Filter articles containing the term "Africa"
search_term = "sécurité"
filtered_articles_secu= df[df['Texte'].str.contains(search_term, case=False)]

# Display the filtered articles
filtered_articles_secu

,Unnamed: 0,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term,Tokens
5,6,Le Monde (site web),Les sept infos non liées au coronavirus que vo...,2020-03-20,Le Monde,On ne peut pas vous reprocher de penser que le...,nde s'est arrêté de tourner à cause de l'épid...,"véritable monde alternatif, serein et chaleur...",Monde,migrant_migratoire,"[peut, reprocher, penser, monde, sest, arrt, t..."
7,8,Le Monde,Télésommet sur Idlib entre les Européens et Er...,2020-03-19,Marie Jégo et Piotr Smolar (à paris) Marie Jé...,Istanbul correspondante - Le coronavirus boule...,"se tout, mais les dirigeants européens ont enc...","mmentaires, en rappelant « l'offensive militai...",Monde,migrant_migratoire,"[istanbul, correspondante, coronavirus, boulev..."
13,14,La Croix (site web),Crise migratoire?: bras de fer autour de la fr...,2020-03-11,Marie Verdier,Après la visite du président turc Recep Tayyip...,"dogan à Bruxelles, lundi 9 mars, Emmanuel Mac...",nement pour faire face à l'afflux migratoire ...,Croix,migrant_migratoire,"[aprs, visite, prsident, turc, recep, tayyip, ..."
17,18,Les Echos (site web),Migrants : l'Europe propose d'accueillir 1.500...,2020-03-09,LES ECHOS,Situation ultra-tendue en Grèce. Lescamps de r...,giés surpeuplés et les conditions de vie de pl...,"de sécurité, de stabilité dans la région et d...",Autre,migrant_migratoire,"[situation, ultratendue, grce, lescamps, rfugi..."
21,22,Le Figaro (site web),La Lettre du Figaro du 6 mars 2020,2020-03-06,"Meteyer, Madeleine","Bonjour très chers abonnés, Ah, la douce pai...",'une vie dans une petite commune de province! ...,prendre des photos dans le métro était interdi...,Figaro,migrant_migratoire,"[bonjour, trs, chers, abonns, ah, douce, paix,..."
...,...,...,...,...,...,...,...,...,...,...,...
8015,8029,Le Parisien.fr,Syrie : Obama s'avoue indécis face à l'Etat i...,2014-08-29,NaN,/international/syrie-obama-s-avoue-indecis-fac...,-l-etat-islamique-29-08-2014-4095593.php Ave...,"s soupà§onnées de liens avec Al-Qaà¯da, et uti...",Parisien,asylum_refugee,[internationalsyrieobamasavoueindecisfacealeta...
8016,8030,Libération,Soutenir la résistance kurde,2014-08-13,Kendal Nezan,L'Etat islamique constitue une menace mortelle...,ur la survie de l'Irak et pour la stabilité de...,ihadistes. L'action aura un coà»t mais celui d...,Libération,asylum_refugee,"[letat, islamique, constitue, menace, mortelle..."
8017,8031,Le Monde,ANALYSE L'histoire est-elle en train de se r...,2014-04-15,NaN,L'information s'est affichée en moins de 140 c...,"ctères : "" Gemayel : les réfugiés menacent l'...",ues au Liban ont presque disparu pour ne plus ...,Monde,asylum_refugee,"[linformation, sest, affiche, moins, caractres..."
8019,8033,Le Figaro.fr,Syrie : un afflux de réfugiés inédit depuis le...,2013-07-17,"Rose, Sunniva","En Syrie, 5000 personnes meurent chaque mois, ...",t plus de 150 par jour. La population syrienne...,"il faudrait construire des vrais camps, avec d...",Figaro,asylum_refugee,"[syrie, personnes, meurent, chaque, mois, plus..."


In [25]:
# Filter articles containing the term "Africa"
search_term = "humanitaire"
filtered_articles_humanitaire= df[df['Texte'].str.contains(search_term, case=False)]

# Display the filtered articles
filtered_articles_humanitaire

,Unnamed: 0,Journal,Titre,Date,Auteur,Texte,extrait_debut,extrait_fin,CJournal,crisis_term,Tokens
0,1,"Le Figaro, no. 23520",L'agenda caché de Moscou en Italie,2020-03-30,"Lasserre, Isabelle",LES SYMBOLES comptent parfois plus que les fai...,"Sur les camions militaires qui, en colonnes, ...",Deux semaines pour se ressaisir et s'unir ou p...,Figaro,migrant_migratoire,"[symboles, comptent, parfois, plus, faits, cam..."
7,8,Le Monde,Télésommet sur Idlib entre les Européens et Er...,2020-03-19,Marie Jégo et Piotr Smolar (à paris) Marie Jé...,Istanbul correspondante - Le coronavirus boule...,"se tout, mais les dirigeants européens ont enc...","mmentaires, en rappelant « l'offensive militai...",Monde,migrant_migratoire,"[istanbul, correspondante, coronavirus, boulev..."
13,14,La Croix (site web),Crise migratoire?: bras de fer autour de la fr...,2020-03-11,Marie Verdier,Après la visite du président turc Recep Tayyip...,"dogan à Bruxelles, lundi 9 mars, Emmanuel Mac...",nement pour faire face à l'afflux migratoire ...,Croix,migrant_migratoire,"[aprs, visite, prsident, turc, recep, tayyip, ..."
14,15,Le Figaro (site web),Syrie: le président turc demande le «soutien c...,2020-03-09,Le Figaro avec AFP,Le président turc Recep Tayyip Erdogan a deman...,à l'Otan son « soutien concret » dans le conf...,"un pays voisin, rejette la faute sur la Turqui...",Figaro,migrant_migratoire,"[prsident, turc, recep, tayyip, erdogan, a, de..."
17,18,Les Echos (site web),Migrants : l'Europe propose d'accueillir 1.500...,2020-03-09,LES ECHOS,Situation ultra-tendue en Grèce. Lescamps de r...,giés surpeuplés et les conditions de vie de pl...,"de sécurité, de stabilité dans la région et d...",Autre,migrant_migratoire,"[situation, ultratendue, grce, lescamps, rfugi..."
...,...,...,...,...,...,...,...,...,...,...,...
8014,8028,Libération (site web),Plus de trois millions de Syriens forcés de fu...,2014-08-29,NaN,Plus de trois millions de Syriens ont fui la g...,re civile qui ravage leur pays et sont devenus...,s partenaires nationaux continuent de fournir ...,Libération,asylum_refugee,"[plus, trois, millions, syriens, fui, guerre, ..."
8016,8030,Libération,Soutenir la résistance kurde,2014-08-13,Kendal Nezan,L'Etat islamique constitue une menace mortelle...,ur la survie de l'Irak et pour la stabilité de...,ihadistes. L'action aura un coà»t mais celui d...,Libération,asylum_refugee,"[letat, islamique, constitue, menace, mortelle..."
8019,8033,Le Figaro.fr,Syrie : un afflux de réfugiés inédit depuis le...,2013-07-17,"Rose, Sunniva","En Syrie, 5000 personnes meurent chaque mois, ...",t plus de 150 par jour. La population syrienne...,"il faudrait construire des vrais camps, avec d...",Figaro,asylum_refugee,"[syrie, personnes, meurent, chaque, mois, plus..."
8022,8036,Le Figaro.fr,Libye/UE: 30M d'euros pour les réfugiés,2011-03-03,AFP,La Commissaire européenne Kristalina Georgieva...,"hargée de la coopération internationale, de l'...",e à Tunis avec le ministre des affaire étrang...,Figaro,asylum_refugee,"[commissaire, europenne, kristalina, georgieva..."
